# Delivery Layout

When you deliver something you'll probably have a layout:

  - a static or dynamic inventory of all the nodes to manage
  - ssh keys to use
  - users and secrets to connect to the hosts
  - whether to do privilege escalation (eg. sudo, ...) before running tasks
  - if nodes should be accessed via a bastion host, docker, ...
  
Put those informations, together with a brief description of the playbook usage (eg. 2/3 lines) into ansible.cfg

![delivery layout](https://cdn.pbrd.co/images/39e3p1vlg.png)


In [1]:
## Simple delivery layout

In [8]:
cd exercise-00

[Errno 2] No such file or directory: 'exercise-00'
/notebooks/exercise-00


# ansible.cfg

When running ansible, the first file read is ansible.cfg, resolved in the following order:

  - `ANSIBLE_CONFIG` (env var)
  - `./ansible.cfg` (in the current directory)
  - `~.ansible.cfg` (in the home directory)
  - `/etc/ansible/ansible.cfg`

We'll create a new ansible.cfg for every project!

In [3]:
# When running ansible, the first file to be read is
!cat ansible.cfg

#
# Always use ansible.cfg and inventory files to 
#  describe your deployment! 
#
# This will save time and helps your colleagues to
#  keep in touch with the environment.
#
[defaults]
# define our inventory file or ansible defaults
#  to /etc/ansible/hosts.
inventory = inventory

# It's just for testing purposes (or if you're on a secure cloud)
#  you may want to avoid typing yes to all host keys.
# Once you get host keys, you can comment this out and re-enable
#  checks.
host_key_checking = False

# When an ansible plabook fails, it creates a retry file which may
#  pollute the current directory ;) You can move this out using
#  further variables
retry_files_enabled = False

# Search password file in here
# vault_password_file = ~/.ssh/pin


# [ssh_connection]
# ssh_args = -F ./ssh_config/config -o ControlMaster=auto -o ControlPersist=30m
# control_path = ./ssh_config/ansible-%%r@%%h:%%p


## Exercise 

 - ping all hosts without specifying an inventory file
 - comment the "inventory" line out of [ansible.cfg](/edit/notebooks/exercise-00/ansible.cfg)
 - try to ping then again

In [6]:
# Use this cell for the exercise

In [9]:
# Solution
!sed -i 's/^inventory/#inventory/' ansible.cfg
!ansible -m ping all
!sed -i 's/#inventory/inventory/' ansible.cfg

 [WARNING]: Host file not found: /etc/ansible/hosts

 [WARNING]: provided hosts list is empty, only localhost is available

 [WARNING]: No hosts matched, nothing to do



## Exercise

You can subscript host groups, eg: `all[0]` is the first host in inventory.

  - ping only the first host
  - then the second

In [12]:
# Solution
!ansible -m ping all[1]

172.17.0.5 | UNREACHABLE! => {
    "changed": false, 
    "msg": "Failed to connect to the host via ssh: Permission denied (publickey,password).\r\n", 
    "unreachable": true
}


In [14]:
# Exercise
# can you find a default ansible.cfg?
# if not, have a look at it on your local distro.



## authentication

You can manage machines via `ssh` or `docker`, but what happens if `PermitRootLogin=no`?

Just use
```
become: yes
become_user: root
become_method: sudo  # defaults to sudo
```

## Exercise

You can specify which ssh key to use: 

  - which parameter allows to set the default ssh identity?



## Inventory

The inventory contains the infrastructure host. Maintaining an inventory helps to:

  - clearly state each host and its functionalities
  - communicate to others all the involved machines
  - describe the infrastructure

Via `ansible.cfg` you can set a default inventory. You could  eg. default to staging  and require `-i production` to run on actual machines.

Ansible supports dynamic inventories (ldap, script, ..) [see inventory chapter](/notebooks/notebooks/04_inventories.ipynb)

## Encrypt secrets

You can use and deliver secrets in your infrastructure using an encrypted file (aka vault).

Decryption password can be typed each time or can be stored in a pin file configured in `ansible.cfg`.

REMEMBER: clear your pin file at logout ;) 

## Bastion

A bastion host is the unique management   entrypoint  for an infrastructure.

![title](https://cloud.google.com/solutions/images/bastion.png)
    
Ansible *leverages ssh functionalities* to manage resources from your local machine thru a bastion.
With a proper configuration you can run your commands/playbooks without continusly moving files to and fro your bastion.

Those includes:
 
  - socks 
  - local and reverse tunnels (ssh -L | -R )
  